In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import dataframe_image as dfi
import matplotlib.pyplot as plt
import imgkit as imgkit
from html2image import Html2Image

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
data = pd.read_csv('Wine_Price_Data.csv')
dfi.export(data.describe(), 'images/original_data.png')
data.describe()

,Vintage,Red,White,Rosé,Orange,Sparkling,Winery?,Vineyard?,Oaked?,AVA,Price/750mL,Rating,ABV %
count,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00
mean,2020.01,0.56,0.40,0.06,0.00,0.04,0.90,0.51,0.68,0.81,27.88,87.92,13.62
std,1.44,0.50,0.49,0.24,0.07,0.19,0.30,0.50,0.47,0.39,29.89,3.06,1.29
min,2014.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.99,72.00,6.80
25%,2019.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00,12.99,86.00,13.00
50%,2021.00,1.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,18.99,88.00,13.80
75%,2021.00,1.00,1.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,33.50,90.00,14.50
max,2022.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,260.99,96.00,16.00


# Cleaning Data

In [4]:
# add log price, dropping producer, making everything uppercase, removing the one orange entry
data['Log_Price'] = np.log(data['Price/750mL'])
data["Age"] = 2023 - data["Vintage"]
data = data.drop(['Producer', 'Price/750mL', "Vintage", "Orange"], axis=1) 
for col in ["Variety/Vine", "State"]:
    data[col] = data[col].str.upper()
    data[col] = data[col].str.strip()
    print(data[col].value_counts())

CABERNET SAUVIGNON                                       36
PINOT NOIR                                               35
CHARDONNAY                                               35
BLEND                                                    28
MERLOT                                                   11
SAUVIGNON BLANC                                          10
PINOT GRIGIO                                              7
PINOT GRIS                                                7
RIESLING                                                  6
RED BLEND                                                 5
ZINFANDEL                                                 5
ROSÉ                                                      4
CABARNET SAUVIGNON                                        3
PETITE SIRAH                                              3
GRENACHE                                                  2
SYRAH                                                     2
MOSCATO                                 

In [5]:
# Cleaning Data, must need at least 3 entries or else it will be classified as other

data["State"] = data["State"].str.replace("NC", "NORTH CAROLINA")

for col in ["Variety/Vine"]:
    data[col] = data[col].str.replace("CHARDONNNAY", "CHARDONNAY")
    data[col] = data[col].str.replace("BUTTERY CHARDONNAY", "CHARDONNAY")
    data[col] = data[col].str.replace("ROSE", "ROSÉ")
    data[col] = data[col].str.replace("FUME BLANC", "SAUVIGNON BLANC")
    data[col] = data[col].str.replace("ORANGE SAUVIGNON BLANC", "SAUVIGNON BLANC")
    data[col] = data[col].str.replace("CABARNET SAUVIGNON", "CABERNET SAUVIGNON")
    data[col] = data[col].str.replace("ZINFANDEL BLANC NOIR", "ZINFANDEL")
    data[col] = data[col].str.replace("PINOT NOIR BLANC", "PINOT NOIR")
    data[col] = data[col].str.replace("JOVINO PINOT NOIR", "PINOT NOIR")
    data[col] = data[col].str.replace("PINO GRIS", "PINOT GRIS")
    for blend in ["CHERRY MOSCATO","RED BLEND", "PINK MOSCATO", "CHAMPAGNE EXTRA DRY", "BLEND: SYRAH, GRENACHE, MOURVEDRE, CABERNET SAUVIGNON"]:
        data[col] = data[col].str.replace(blend, "BLEND")
    unique_entries = data[col].value_counts()
    
print(data["State"].value_counts())
print(data["Variety/Vine"].value_counts())


CALIFORNIA        158
OREGON             26
WASHINGTON         15
NORTH CAROLINA     11
NEW YORK            4
WASHINGOTN          2
INDIANA             1
NEVADA              1
WEST VIRGINIA       1
Name: State, dtype: int64
CABERNET SAUVIGNON    39
BLEND                 37
CHARDONNAY            37
PINOT NOIR            36
SAUVIGNON BLANC       12
MERLOT                11
PINOT GRIS             8
PINOT GRIGIO           7
RIESLING               6
ZINFANDEL              6
ROSÉ                   5
PETITE SIRAH           3
MOSCATO                2
SYRAH                  2
GRENACHE               2
GREEN APPLE WINE       1
NIAGARA                1
MUSCADINE              1
VIOGNEIR               1
SCUPPERNONG            1
CARIGNAN               1
Name: Variety/Vine, dtype: int64


In [6]:
# add other for states and variety/vine with less than 3 entries
for col in ["Variety/Vine", "State"]:
    print(f"Break Down of Other for {col}")

    unique_entries = data[col].value_counts()

    data_frame_other = pd.DataFrame(unique_entries[unique_entries < 3])
    data_frame_other_variety = data_frame_other.to_dict()[col].keys()
    data_frame_other_count = data_frame_other.to_dict()[col].values()
    data_frame_other_table = pd.DataFrame(data={col: data_frame_other_variety, "Count": data_frame_other_count})
    print(data_frame_other_table)

    dfi.export(data_frame_other_table,  f"images/variety_other_breakdown.png") if col == "Variety/Vine" else dfi.export(data_frame_other_table,  f"images/{col}_other_breakdown.png")

    # changing the data
    data[col] = data[col].apply(lambda x: x if x in unique_entries[unique_entries >= 3].index else "OTHER")
    print(data[col].value_counts())

Break Down of Other for Variety/Vine
       Variety/Vine  Count
0           MOSCATO      2
1             SYRAH      2
2          GRENACHE      2
3  GREEN APPLE WINE      1
4           NIAGARA      1
5         MUSCADINE      1
6          VIOGNEIR      1
7       SCUPPERNONG      1
8          CARIGNAN      1
CABERNET SAUVIGNON    39
BLEND                 37
CHARDONNAY            37
PINOT NOIR            36
OTHER                 12
SAUVIGNON BLANC       12
MERLOT                11
PINOT GRIS             8
PINOT GRIGIO           7
ZINFANDEL              6
RIESLING               6
ROSÉ                   5
PETITE SIRAH           3
Name: Variety/Vine, dtype: int64
Break Down of Other for State
           State  Count
0     WASHINGOTN      2
1        INDIANA      1
2         NEVADA      1
3  WEST VIRGINIA      1
CALIFORNIA        158
OREGON             26
WASHINGTON         15
NORTH CAROLINA     11
OTHER               5
NEW YORK            4
Name: State, dtype: int64


In [7]:
# One Hot Encoding
data = pd.get_dummies(data, columns=["Variety/Vine", "State"], drop_first=False)
dfi.export(data.describe().T, 'images/cleaned_data.png')
data.drop(["Variety/Vine_SAUVIGNON BLANC", "State_WASHINGTON", "White"], axis=1, inplace=True)

# Cleaned Up Data Summary and Analysis

In [8]:
data.describe()

,Red,Rosé,Sparkling,Winery?,Vineyard?,Oaked?,AVA,Rating,ABV %,Log_Price,...,Variety/Vine_PINOT GRIS,Variety/Vine_PINOT NOIR,Variety/Vine_RIESLING,Variety/Vine_ROSÉ,Variety/Vine_ZINFANDEL,State_CALIFORNIA,State_NEW YORK,State_NORTH CAROLINA,State_OREGON,State_OTHER
count,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,...,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00,219.00
mean,0.56,0.06,0.04,0.90,0.51,0.68,0.81,87.92,13.62,3.04,...,0.04,0.16,0.03,0.02,0.03,0.72,0.02,0.05,0.12,0.02
std,0.50,0.24,0.19,0.30,0.50,0.47,0.39,3.06,1.29,0.69,...,0.19,0.37,0.16,0.15,0.16,0.45,0.13,0.22,0.32,0.15
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72.00,6.80,1.38,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,1.00,0.00,0.00,1.00,86.00,13.00,2.56,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,1.00,0.00,0.00,1.00,1.00,1.00,1.00,88.00,13.80,2.94,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
75%,1.00,0.00,0.00,1.00,1.00,1.00,1.00,90.00,14.50,3.51,...,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,96.00,16.00,5.56,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [9]:
# print all the data
data.to_csv('Wine_Price_Data_Cleaned.csv', index=False)

# Data Analysis

In [10]:
cleaned_data = pd.read_csv('Wine_Price_Data_Cleaned.csv')
sm.add_constant(cleaned_data)
Y = cleaned_data["Log_Price"]
X = data.drop(['Log_Price'], axis=1)
X = sm.add_constant(X)

In [11]:
model = sm.OLS(Y, X).fit()
print(model.summary())

# need to use an html to image converter to get the summary
f = open("images/summary.html", "w")
f.write(model.summary().as_html())
f.close()


                            OLS Regression Results                            
Dep. Variable:              Log_Price   R-squared:                       0.585
Model:                            OLS   Adj. R-squared:                  0.526
Method:                 Least Squares   F-statistic:                     9.974
Date:                Tue, 30 May 2023   Prob (F-statistic):           3.42e-24
Time:                        17:34:35   Log-Likelihood:                -133.18
No. Observations:                 219   AIC:                             322.4
Df Residuals:                     191   BIC:                             417.2
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

makes since that vintage is negative since higher the vintage (younger the wine) price deacreases by .38% and it was find that in the stuyd 4.8% decrease, which is off. 

# interperting data, saugvginon blanc is dropped and so was WAshington


# Example Prediction using 

In [12]:
# predict using the model
log_actual_price = np.log(60.99)
data = np.array([1, # constant
                 1,0,0, # red
                 1, 1, # winery, vineyard
                 1, # oaked
                 1, # ava status
                 91, # Rating
                 14.7, #ABV
                 3, # Age
                 0, 1, 0,0,0,0,0,0,0,0,0,0, # Variety/Vine
                 1,0,0,0,0]) # State

print(f"Estimated log(price)= {data.dot(model.params.values)}")
print("Actual log(price)= " + str(log_actual_price))

Estimated log(price)= 3.7546425842764846
Actual log(price)= 4.110709916308365
